In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
def gen_synthetic(n,d, sigma):
    
    X = np.random.normal(0, 1, n*d).reshape(n, d)
    w = np.random.normal(0, 1, d)
    y = X@w + np.random.normal(0, sigma, n)
    
    return X, y, w

In [30]:
n=500
d=20
X, y, w_star = gen_synthetic(n, d, 0.1)

In [31]:
w_star

array([ 1.88267651,  2.14703697,  0.42124238,  0.41378537, -0.94480181,
        1.21295239,  2.19898558,  0.09852006, -0.80967307, -1.17435525,
        0.13828755,  0.13890834, -1.11108905, -0.59207476, -0.38563503,
       -1.87063353, -0.03429733, -0.03571881, -0.73484923, -0.71576944])

In [63]:
w = np.linalg.pinv(X)@y
w

array([ 1.88954263,  2.1551777 ,  0.42342156,  0.41625003, -0.94707039,
        1.21257269,  2.20460825,  0.09154577, -0.81045735, -1.17647297,
        0.13950721,  0.13233538, -1.11844949, -0.58729595, -0.38777008,
       -1.87279675, -0.02444276, -0.0292303 , -0.73265009, -0.71319645])

In [68]:
np.linalg.norm(X@w-y)**2, np.linalg.norm(np.abs(X@w-y) + np.linalg.norm(w)*np.ones(n))**2

(4.6720013740729085, 12481.666560814387)

# Robust stuff 

In [74]:
def rob_linreg(X,y, rho, lambda_):
    # data, labels = X,y
    n,d = X.shape
    w_rob = cp.Variable(d)

    abs_resids = cp.abs(X@w_rob - y)
    robper = rho*cp.norm(w_rob, p=2)*np.ones(n)
    ridge_pen = lambda_*cp.norm(w_rob, p=2)**2
    obj = cp.norm(abs_resids + robper, p=2)**2 + ridge_pen

    prob = cp.Problem(cp.Minimize(obj))
    prob.solve()
    return w_rob, prob

w_rob, prob = rob_linreg(X, y, 1, 0)
print(prob.value)

10642.551074225774


In [73]:
np.linalg.norm(X@w_rob.value-y)**2

3355.168983921537

In [69]:
w_rob.value

array([ 0.83059411,  0.9904502 ,  0.1863713 ,  0.16980354, -0.47383208,
        0.64626007,  0.96508304,  0.05305684, -0.3193917 , -0.53284632,
        0.01256961, -0.06820422, -0.51828928, -0.28408702, -0.27768418,
       -0.86685609, -0.06954154,  0.08101882, -0.38469031, -0.27445241])

In [62]:
w_star

array([ 1.88267651,  2.14703697,  0.42124238,  0.41378537, -0.94480181,
        1.21295239,  2.19898558,  0.09852006, -0.80967307, -1.17435525,
        0.13828755,  0.13890834, -1.11108905, -0.59207476, -0.38563503,
       -1.87063353, -0.03429733, -0.03571881, -0.73484923, -0.71576944])